In [22]:
import pandas as pd
import mxnet
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split

#=========================================================#

##나중에 argpaser로 변경
max_grad_norm = 1
log_interval = 1000
warmup_ratio = 0.1
num_epochs = 5
batch_size = 4
max_len = 512
learning_rate = 5e-5 

#########################



device = torch.device("cuda:1")
print(f"Using {device}")


data = pd.read_pickle("../result/sample5_tokenized.pkl")
##############

print(data.info())

label_to_int = {}
for i, item in enumerate(data['접수기관'].unique()):
    label_to_int[item] = i

data['접수기관'] = data['접수기관'].apply(lambda x : label_to_int[x])
data = data[['token', '접수기관']]



dataset_train, dataest_test = train_test_split(data, test_size=0.1, random_state=42)


print(f"train len : {len(dataset_train)}, test len : {len(dataest_test)}")



class BERTDataset(Dataset):
    def __init__(self, dataset,bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([" ".join(dataset.iloc[i]['token'])]) for i in range(len(dataset))]
        self.labels = [np.int32(dataset.iloc[i]['접수기관']) for i in range(len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))



print('get bertmodel and vocab')
bertmodel, vocab = get_pytorch_kobert_model()


print("data setting")
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

train_data = BERTDataset(dataset_train, tok, max_len, True, False)
test_data = BERTDataset(dataest_test, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(
    train_data, batch_size = batch_size, num_workers = 8)

test_dataloader = torch.utils.data.DataLoader(
    test_data, batch_size = batch_size, num_workers = 8)


class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = len(dataset_train['접수기관'].unique()),   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


    #BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader


print("Train Start")
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} / {} loss {} train acc {}".format(e+1, batch_id+1 , len(train_dataloader), loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))



Using cuda:1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   제목      1000000 non-null  object
 1   본문      999751 non-null   object
 2   민원발생지   999972 non-null   object
 3   접수기관    1000000 non-null  object
 4   token   1000000 non-null  object
dtypes: object(5)
memory usage: 38.1+ MB
None
train len : 900000, test len : 100000
get bertmodel and vocab
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
data setting
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Train Start


  0%|          | 0/225000 [00:00<?, ?it/s]

epoch 1 batch id 1 / 225000 loss 5.657588005065918 train acc 0.0
epoch 1 batch id 101 / 225000 loss 5.795995712280273 train acc 0.0
epoch 1 batch id 201 / 225000 loss 5.622037410736084 train acc 0.0012437810945273632
epoch 1 batch id 301 / 225000 loss 5.938326358795166 train acc 0.0008305647840531562
epoch 1 batch id 401 / 225000 loss 5.917154312133789 train acc 0.0006234413965087282
epoch 1 batch id 501 / 225000 loss 5.962574005126953 train acc 0.0014970059880239522
epoch 1 batch id 601 / 225000 loss 6.045806884765625 train acc 0.0016638935108153079
epoch 1 batch id 701 / 225000 loss 5.707292079925537 train acc 0.001783166904422254
epoch 1 batch id 801 / 225000 loss 5.83020544052124 train acc 0.0024968789013732834
epoch 1 batch id 901 / 225000 loss 5.764742851257324 train acc 0.00388457269700333
epoch 1 batch id 1001 / 225000 loss 5.72004508972168 train acc 0.004245754245754246
epoch 1 batch id 1101 / 225000 loss 5.554431915283203 train acc 0.004087193460490463
epoch 1 batch id 1201 /

  0%|          | 0/25000 [00:00<?, ?it/s]

epoch 1 test acc 0.74562


...

epoch 4 test acc 0.79426


  0%|          | 0/225000 [00:00<?, ?it/s]

epoch 5 batch id 1 / 225000 loss 1.9072415828704834 train acc 0.5
epoch 5 batch id 101 / 225000 loss 1.7172784805297852 train acc 0.8118811881188119
epoch 5 batch id 201 / 225000 loss 1.7981948852539062 train acc 0.8121890547263682
epoch 5 batch id 301 / 225000 loss 0.4723477065563202 train acc 0.8156146179401993
epoch 5 batch id 401 / 225000 loss 0.044587306678295135 train acc 0.807356608478803
epoch 5 batch id 501 / 225000 loss 1.070417046546936 train acc 0.8073852295409182
epoch 5 batch id 601 / 225000 loss 1.9234647750854492 train acc 0.8049084858569051
epoch 5 batch id 701 / 225000 loss 1.8338991403579712 train acc 0.7999286733238231
epoch 5 batch id 801 / 225000 loss 1.990224838256836 train acc 0.8043071161048689
epoch 5 batch id 901 / 225000 loss 0.05950021743774414 train acc 0.8002219755826859
epoch 5 batch id 1001 / 225000 loss 1.0702122449874878 train acc 0.8044455544455544
epoch 5 batch id 1101 / 225000 loss 0.0009270061855204403 train acc 0.8054041780199819
epoch 5 batch id

  0%|          | 0/25000 [00:00<?, ?it/s]

epoch 5 test acc 0.79898


In [23]:
PATH = '../result/kobert/' # google 드라이브 연동 해야함. 관련코드는 뺐음
torch.save(model, PATH + 'KoBERT_0623_e5.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'Kobert_0623_e5_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar') 

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
Komodel = torch.load(PATH + 'KoBERT_0623_e5.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
Komodel.load_state_dict(torch.load(PATH + 'Kobert_0623_e5_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

<All keys matched successfully>

In [14]:
_, dataset_Q = train_test_split(data, test_size=0.01, random_state=42)

#bertmodel, vocab = get_pytorch_kobert_model()

print("data setting")
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

Q_data = BERTDataset(dataset_Q, tok, max_len, True, False)

Q_dataloader = torch.utils.data.DataLoader(
    Q_data, batch_size = 4, num_workers = 8)

using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
data setting
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [20]:
dataset_Q

,token,접수기관
260189,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 광주, 광역시, 광산구, 쌍,...",37
969141,"[불법, 주차, 울산, 광역시, 남구, 신정동, 부근, 신고, 위치, 안내, 지방행...",21
573889,"[평소, 업무, 협조, 감사, 당사, 지자체, 로부터, 생활, 폐기물, 발생, 음식...",63
32931,"[불법, 정차, 불법, 정차, 구역, 정차, 통행, 방해, 함, 상세, 위치, 설명...",40
739352,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 경기도, 용인시, 처인구, 역...",19
...,...,...
88065,"[통행, 사진, 차, 주말, 양옆, 가운데, 주차, 사람, 통행, 불편, 끼, 기,...",60
359477,"[장애인, 전용, 구역, 불법, 주차, 홈플러스, 장애인, 구역, 불법, 정차, 경...",40
887074,"[민원, 유입, 경로, 모바일, 사고, 발생, 지역, 충청남도, 서천군, 장항읍, ...",311
164007,"[번호판, 양쪽, 검은색, 스티커, 부착, 차량]",85


In [15]:
for item in Q_dataloader:
    print(i)
    break

330


In [19]:
Komodel.eval()
test_acc = 0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(Q_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = Komodel(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/250 [00:00<?, ?it/s]

epoch 5 test acc 0.776
